In [1]:
#import geopandas as gpd
import pandas as pd
import numpy as np
import json  as js
#from geopandas import GeoDataFrame
#from shapely.geometry import Point
#import shapely
from datetime import datetime as dt
import csv

In [2]:
#path='C:\\Users\\makol\\Documents\\Python_Scripts\\Samara_mobile\\CEI_report_v_3_20200915\\with_2G\\'

In [3]:
aglom = pd.read_csv('STA.csv', sep = ';', encoding='utf8')

In [2]:
b = pd.read_csv('report_f_all_v3.csv', sep = ';', encoding='utf8')

In [5]:
len(b)

13832242

In [11]:
b.head()

,time_key,ts,zid_from,zid_to,15_min_ind,30_min_ind,60_min_ind,90_min_ind,buffer_out_ind,3_cells_ind,count_ctn
0,2019-02-07,2019-02-07 03:00:00,1,6,1,1,1,1,1,0,2
1,2019-02-07,2019-02-07 03:00:00,1,23,1,1,1,0,1,0,1
2,2019-02-07,2019-02-07 03:00:00,1,25,1,1,1,1,1,0,1
3,2019-02-07,2019-02-07 03:00:00,1,31,1,1,1,1,1,0,1
4,2019-02-07,2019-02-07 03:00:00,1,62,1,0,0,0,1,0,1


In [6]:
set_id=aglom['zid_from'].unique()

In [3]:
b2=b[b['60_min_ind']==1]
b2=b2[b2['buffer_out_ind']==1]
#b2=b2[b2['zid_from'].isin(set_id)]
#b2=b2[b2['zid_to'].isin(set_id)]

In [14]:
len(b2)

6453674

In [15]:
b2['time_key'].unique()

array(['2019-02-07', '2019-02-08', '2019-02-09', '2019-04-11',
       '2019-04-12', '2019-04-13', '2019-07-11', '2019-07-12',
       '2019-07-13', '2019-10-17', '2019-10-18', '2019-10-19'],
      dtype=object)

In [4]:
friday=b2[b2['time_key'].isin(['2019-02-08','2019-04-12','2019-07-13','2019-10-18'])]
thuesday=b2[b2['time_key'].isin(['2019-02-07','2019-04-11','2019-07-11','2019-10-17'])]
sat=b2[b2['time_key'].isin(['2019-02-09','2019-04-13','2019-07-13','2019-10-19'])]

In [5]:
thuesday_morn=thuesday[thuesday['ts'].str.contains(' 07:30')]\
        .append(thuesday[thuesday['ts'].str.contains(' 08:')])\
        .append(thuesday[thuesday['ts'].str.contains(' 09:')])\
        .append(thuesday[thuesday['ts'].str.contains(' 10:00')])

In [6]:
thuesday_morn['ts'].unique()

array(['2019-02-07 07:30:00', '2019-04-11 07:30:00',
       '2019-07-11 07:30:00', '2019-10-17 07:30:00',
       '2019-02-07 08:00:00', '2019-02-07 08:30:00',
       '2019-04-11 08:00:00', '2019-04-11 08:30:00',
       '2019-07-11 08:00:00', '2019-07-11 08:30:00',
       '2019-10-17 08:00:00', '2019-10-17 08:30:00',
       '2019-02-07 09:00:00', '2019-02-07 09:30:00',
       '2019-04-11 09:00:00', '2019-04-11 09:30:00',
       '2019-07-11 09:00:00', '2019-07-11 09:30:00',
       '2019-10-17 09:00:00', '2019-10-17 09:30:00',
       '2019-02-07 10:00:00', '2019-04-11 10:00:00',
       '2019-07-11 10:00:00', '2019-10-17 10:00:00'], dtype=object)

In [7]:
zid_total_morn=pd.DataFrame(thuesday_morn.groupby(['time_key','zid_from','zid_to']).agg({'count_ctn': np.sum})).reset_index()

In [8]:
total_morn=pd.DataFrame(zid_total_morn.groupby(['zid_from','zid_to']).agg({'count_ctn': np.mean})).reset_index()

In [9]:
total_morn['morn_count_ctn']=total_morn['count_ctn'].apply(np.ceil)
total_morn=total_morn.drop(['count_ctn'], axis=1)

In [19]:
zid_total=pd.DataFrame(thuesday.groupby(['time_key','zid_from','zid_to']).agg({'count_ctn': np.sum})).reset_index()
total=pd.DataFrame(zid_total.groupby(['zid_from','zid_to']).agg({'count_ctn': np.mean})).reset_index()
total['all_day_count_ctn']=total['count_ctn'].apply(np.ceil)
total=total.drop(['count_ctn'], axis=1)

In [20]:
prop = pd.read_csv('prop2.csv',sep = ';', encoding='utf-8')
prop['zid_from']=prop['Номер транспортного района']
prop['zid_to']=prop['Номер транспортного района']
prop=prop.drop(['Номер транспортного района'],axis=1)
prop=prop.drop(['Муниципалитет (нижний уровень)'],axis=1)
prop=prop.drop(['Город/ Муниципалитет'],axis=1)
prop=prop.drop(['Км от 156 района'],axis=1)

In [21]:
total=total.merge(total_morn,on = ['zid_from','zid_to'],how='outer')

In [22]:
total=total.fillna(0)

In [23]:
total=total.merge(prop.drop('zid_to',axis=1),on='zid_from',how='left')

In [24]:
total

,zid_from,zid_to,all_day_count_ctn,morn_count_ctn,Пригород чего
0,1,2,15.0,3.0,городской округ Самара
1,1,3,53.0,9.0,городской округ Самара
2,1,4,61.0,11.0,городской округ Самара
3,1,5,3.0,1.0,городской округ Самара
4,1,6,385.0,60.0,городской округ Самара
...,...,...,...,...,...
99504,557,549,1.0,0.0,пригород Самары
99505,557,550,1.0,0.0,пригород Самары
99506,557,551,1.0,0.0,пригород Самары
99507,557,553,1.0,0.0,пригород Самары


In [25]:
total=total.merge(prop.drop('zid_from',axis=1),on='zid_to',how='left')

In [28]:
total.to_csv('Beeline_area.csv',sep=';',index=False,encoding='windows-1251')

In [25]:
stz=pd.read_csv('STZ.csv',sep = ';', encoding='windows-1251')

In [26]:
stz.head()

,zid_from,zid_to,id_2,TR_type,POP_FROM,POP_TO,WORK_TO,STUD_TO,STZ_1,STZ_2,STZ_3,STZ_4,STZ_5,TC_from,TC_to,TC-TC,TC_GR
0,1,3,1_3,8,8 131,20 729,17 411,954,469,442,399,397,439,0,4 708,4 708,"1,02"
1,1,4,1_4,3,8 131,1 030,1 628,0,367,407,470,473,411,0,0,0,"1,00"
2,1,6,1_6,3,8 131,69 971,11 292,988,340,370,418,421,374,0,5 498,5 498,"1,02"
3,1,7,1_7,3,8 131,895,179,0,317,402,539,545,413,0,939,939,"1,00"
4,1,8,1_8,3,8 131,3 506,928,0,294,332,393,396,337,0,0,0,"1,00"


In [52]:
total_all=total.merge(stz,on = ['zid_from','zid_to'])

In [53]:
len(total_all)

4956

In [54]:
Sosedi=pd.read_csv('Sosedi.csv',sep = ';', encoding='windows-1251')

In [55]:
pares=Sosedi['pares'].unique()

In [56]:
#total_all['pares']=total_all['zid_from'].astype(str)+'_'+total_all['zid_to'].astype(str)

In [57]:
#total_all=total_all[~total_all['pares'].isin(pares)]

In [58]:
len(total_all)

4956

In [59]:
ID=pd.read_csv('ID_NEW.csv',sep = ';', encoding='windows-1251')

In [60]:
ID['zid_from']=ID['zid_from'].astype(int)

In [61]:
total_new_id=total_all.merge(ID,on = 'zid_from').rename(columns={"ID": "zid_from_new", "zid_to_x": "zid_to"})

In [62]:
total_new_id=total_new_id.drop(['zid_from','zid_to_y'],axis=1)

In [63]:
total_new_id=total_new_id.merge(ID,on = 'zid_to')
total_new_id['zid_to_new']=total_new_id['ID']

In [64]:
total_new_id['zid_from'].unique()

array([  3,   4,   6,   7,   8,   9,  10,  12,  13,  14,  16,  22,  23,
        24,  25,  31,  34,  36,  37,  45,  49,  51,  71,  72,  75,  76,
        81, 120, 130, 145, 146, 156, 161, 162, 163, 166, 228, 278, 296,
       301, 302, 303, 305, 310, 336, 479, 485, 532, 535, 537, 538, 546,
       550, 143, 147, 148, 153, 154, 155, 160, 186, 229, 276, 519, 543,
       544, 547, 553,   1,   2, 144, 157, 158, 164, 165, 181, 227, 277,
       539, 541, 542, 545, 549, 551, 552, 555, 556,  18,  19,  20,  21,
        26,  27,  11,  15,  17,  28,  32,  33,  38,  40,  42,  43,  44,
        46, 125, 293, 127, 131,  30,  41, 494,   5,  39, 128, 129, 493,
       495,  35, 496, 140, 119, 126, 132, 274, 299, 540, 554, 509, 188,
       124, 150, 223, 259, 478, 517, 151, 159, 224, 230, 515, 518, 522,
       516, 149, 226, 524, 511, 512, 513, 514, 510, 548, 523, 123, 204,
       225, 152, 231, 275, 525, 258, 477, 520, 521, 295, 300, 475])

In [65]:
total_new_id_sum=pd.DataFrame(total_new_id.groupby(['zid_from_new','zid_to_new'])\
                              .agg({'all_day_count_ctn': np.sum,'morn_count_ctn': np.sum,\
                                    'STZ_1': np.mean,'STZ_2': np.mean,'STZ_3': np.mean,'STZ_4': np.mean,'STZ_5': np.mean})).reset_index()

In [66]:
total_new_id_sum=total_new_id_sum[total_new_id_sum['all_day_count_ctn']>49]

In [70]:
pares2=pd.read_csv('without_pares.csv',sep = ';', encoding='windows-1251')
pares=pares2['pare'].unique()

In [71]:
total_new_id_sum['pares']=total_new_id_sum['zid_from_new'].astype(str)+'_'+total_new_id_sum['zid_to_new'].astype(str)

In [72]:
total_new_id_sum2=total_new_id_sum[~total_new_id_sum['pares'].isin(pares)]

In [73]:
total_new_id_sum2.to_csv('Mobile_for_Model.csv')

In [176]:
proverka = thuesday[(thuesday['zid_from'].isin(Samara))&(thuesday['zid_to'].isin(Samara))]

In [177]:
proverka.to_csv('Tol_Samara.csv')

In [34]:
stz=pd.read_csv('STZ.csv',sep = ';', encoding='windows-1251')

In [35]:
stz

,zid_from,zid_to,STZ_1,STZ_2,STZ_3,STZ_4,STZ_5,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,1,2,484,469,445,444,467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,469,442,399,397,439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,4,367,407,470,473,411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,7,317,402,539,545,413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,8,294,332,393,396,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68901,218,349,1005,1034,1081,1083,1038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68902,218,475,821,845,883,885,848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68903,218,521,755,777,812,813,779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68904,218,213,384,395,413,414,397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
ID=pd.read_csv('ID_NEW.csv',sep = ';', encoding='windows-1251')

In [37]:
stz=stz.merge(ID,on='zid_from')

In [38]:
stz=stz.merge(ID,left_on='zid_to_x',right_on='zid_to')

In [39]:
stz=stz.groupby(['ID_x','ID_y']).agg({'STZ_1': np.mean,'STZ_2': np.mean,'STZ_3': np.mean,'STZ_4': np.mean,'STZ_5': np.mean,}).reset_index()

In [40]:
stz.to_csv('res.csv')